In [4]:
import pandas as pd, numpy as np

train_df = pd.read_csv('data/playground-series-s5e11/train.csv')
test_df = pd.read_csv('data/playground-series-s5e11/test.csv')
orig = pd.read_csv('data/playground-series-s5e11/loan_dataset_20000.csv')

print('Train Shape:', train_df.shape)
print('Test Shape:', test_df.shape)

Train Shape: (593994, 13)
Test Shape: (254569, 12)


In [7]:
train_df.head

<bound method NDFrame.head of             id  annual_income  debt_to_income_ratio  credit_score  \
0            0       29367.99                 0.084           736   
1            1       22108.02                 0.166           636   
2            2       49566.20                 0.097           694   
3            3       46858.25                 0.065           533   
4            4       25496.70                 0.053           665   
...        ...            ...                   ...           ...   
593989  593989       23004.26                 0.152           703   
593990  593990       35289.43                 0.105           559   
593991  593991       47112.64                 0.072           675   
593992  593992       76748.44                 0.067           740   
593993  593993       48959.52                 0.096           752   

        loan_amount  interest_rate  gender marital_status education_level  \
0           2528.42          13.67  Female         Single     Hi

In [5]:
TARGET = 'loan_paid_back'  #boolean
CATS = ['gender', 'marital_status', 'education_level', 'employment_status', 'loan_purpose', 'grade_subgrade']
BASE = [col for col in train_df.columns if col not in ['id', TARGET]]

train = train_df.copy()
test = test_df.copy()

In [6]:
ORIG = []

for col in BASE:
    mean_map = orig.groupby(col)[TARGET].mean()
    new_mean_col_name = f"orig_mean_{col}"
    mean_map.name = new_mean_col_name
    
    train = train.merge(mean_map, on=col, how='left')
    test = test.merge(mean_map, on=col, how='left')
    ORIG.append(new_mean_col_name)

    new_count_col_name = f"orig_count_{col}"
    count_map = orig.groupby(col).size().reset_index(name=new_count_col_name)
    
    train = train.merge(count_map, on=col, how='left')
    test = test.merge(count_map, on=col, how='left')
    ORIG.append(new_count_col_name)

print(len(ORIG), 'Orig Features Created!!')

FEATURES = BASE + ORIG
print(len(FEATURES), 'Total Features.')

22 Orig Features Created!!
33 Total Features.


In [9]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    train[FEATURES], train[TARGET].astype(int), 
    test_size=0.2, random_state=42, stratify=train[TARGET]
)

print(f'Train split (with ORIG features): {X_train.shape}')
print(f'Val split (with ORIG features): {X_val.shape}')

Train split (with ORIG features): (475195, 33)
Val split (with ORIG features): (118799, 33)


In [ ]:
# numeric correlations
numeric_cols = train[BASE].select_dtypes(include=[np.number]).columns.tolist()
numeric_corr = pd.Series(dtype=float)
if numeric_cols:
    numeric_corr = train[numeric_cols].corrwith(train[TARGET].astype(int)).sort_values(ascending=False)

# categorical associations (Cramer's V)
cat_cols = [c for c in BASE if c not in numeric_cols]

# try to use scipy for chi2_contingency; fallback to a safe zero if unavailable
try:
    from scipy.stats import chi2_contingency
    def cramers_v(x, y):
        confusion = pd.crosstab(x, y)
        if confusion.size == 0:
            return 0.0
        chi2 = chi2_contingency(confusion)[0]
        n = confusion.sum().sum()
        if n == 0:
            return 0.0
        phi2 = chi2 / n
        r, k = confusion.shape
        # bias correction
        phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
        rcorr = r - ((r-1)**2)/(n-1)
        kcorr = k - ((k-1)**2)/(n-1)
        denom = min((kcorr-1), (rcorr-1))
        return (phi2corr / denom)**0.5 if denom > 0 else 0.0
except Exception:
    def cramers_v(x, y):
        # fallback: compute association ratio by encoding categories to codes and using Pearson
        x_codes = x.astype('category').cat.codes
        return abs(x_codes.corr(y.astype(int)))

cat_assoc = {}
for c in cat_cols:
    col = train[c].fillna('NA')
    cat_assoc[c] = cramers_v(col, train[TARGET].astype(int))

cat_series = pd.Series(cat_assoc).sort_values(ascending=False)

# Display summaries
print('Numeric features (Pearson correlation with target):')
if not numeric_corr.empty:
    print(numeric_corr.to_string())
else:
    print('  No numeric features found in BASE.')

print('\nTop categorical associations (Cramer\'s V or fallback):')
if not cat_series.empty:
    print(cat_series.head(20).to_string())
else:
    print('  No categorical features found in BASE.')

# combined ranking by absolute strength
combined = pd.concat([
    numeric_corr.abs().rename('strength'),
    cat_series.abs().rename('strength')
]).sort_values(ascending=False)

print('\nTop features by absolute association strength with the target:')
print(combined.head(20).to_string())
